In [1]:
'''
Source: https://medium.datadriveninvestor.com/scraping-live-stock-fundamental-ratios-news-and-more-with-python-a716329e0493

'''

'\nSource: https://medium.datadriveninvestor.com/scraping-live-stock-fundamental-ratios-news-and-more-with-python-a716329e0493\n\n'

In [2]:
import requests
import pandas as pd
from lxml import html
import lxml.html
from lxml import etree
from bs4 import BeautifulSoup as soup
from bs4 import SoupStrainer
from urllib.request import urlopen, Request

In [3]:
symbol='aapl'
url = ("https://finviz.com/news.ashx?t=/forms/post" + symbol.lower())
r = requests.get(url, headers={'User-Agent': 'Mozilla/5.0'})
content = r.content

In [4]:
tree = html.fromstring(content)
tree

<Element html at 0x7fb2c9342dd0>

In [7]:
for element in tree.iter('input'):
     print("Element: %s \n\tvalues(): %s \n\tattrib: %s \n\titems(): %s \n\tkeys(): %s"%
     (element.tag, element.values(),element.attrib,element.items(),element.keys()))
     print("\n")

Element: input 
	values(): ['Search ticker, company or profile', 'text', ''] 
	attrib: {'placeholder': 'Search ticker, company or profile', 'type': 'text', 'value': ''} 
	items(): [('placeholder', 'Search ticker, company or profile'), ('type', 'text'), ('value', '')] 
	keys(): ['placeholder', 'type', 'value']


